In [15]:
!pip install tqdm

  Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)


In [16]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas
from tqdm import tqdm

# Solution 'à l'arrache'

In [12]:
def extract_page(url):
    page_sc = urlopen(url)
    soup = BeautifulSoup(page_sc, 'html.parser')
    livre = soup.find_all(name = 'span', attrs={'itemprop': 'name'})
    titre = []
    auteur = []
    rates = []
    for i, element in enumerate(livre):
        if i % 2 != 0:
            auteur.append(element.text)
        else:
            titre.append(element.text)
    rating = soup.find_all(name = 'span', attrs={'class': 'minirating'})
    for el in rating:
        if el.text.split(' ')[0] == 'really':
            rates.append(el.text.split(' ')[3])
        else:
            rates.append(el.text.split(' ')[1])
    return titre, auteur, rates

In [17]:
global_title = []
global_author = []
global_rates = []

url_base = 'https://www.goodreads.com/list/show/1.Best_Books_Ever'
for i in tqdm(range(1,101)):
    url_complete = url_base + '?page={}'.format(str(i))
    titre, auteur, rates = extract_page(url_complete)
    for title, author, rating in zip(titre, auteur, rates):
        global_title.append(title)
        global_author.append(author)
        global_rates.append(rating)

100%|█████████████████████████████████████████| 100/100 [07:48<00:00,  4.69s/it]


In [19]:
dataframe = pandas.DataFrame(list(zip(global_title, global_author, global_rates)), columns=['Title', 'Author', 'Rate'])

In [20]:
dataframe

,Title,Author,Rate
0,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,4.32
1,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling,4.50
2,To Kill a Mockingbird,Harper Lee,4.27
3,Pride and Prejudice,Jane Austen,4.27
4,"Twilight (The Twilight Saga, #1)",Stephenie Meyer,3.62
...,...,...,...
9995,I Am Princess X,Cherie Priest,4.07
9996,Hate Notes,Vi Keeland,4.14
9997,The Know-It-All: One Man's Humble Quest to Bec...,A.J. Jacobs,3.74
9998,"Twisted Pride (The Camorra Chronicles, #3)",Cora Reilly,4.25


c'est long oui :D

# Solution avec moins de variable
Nous allons donner nos tableaux à la fonction afin qu'elle les alimente directement sans passer par un tableau intermédiaire que nous devons reparcourir

In [21]:
def extract_page2(url, titre, auteur, rates):
    page_sc = urlopen(url)
    soup = BeautifulSoup(page_sc, 'html.parser')
    livre = soup.find_all(name = 'span', attrs={'itemprop': 'name'})
    for i, element in enumerate(livre):
        if i % 2 != 0:
            auteur.append(element.text)
        else:
            titre.append(element.text)
    rating = soup.find_all(name = 'span', attrs={'class': 'minirating'})
    for el in rating:
        if el.text.split(' ')[0] == 'really':
            rates.append(el.text.split(' ')[3])
        else:
            rates.append(el.text.split(' ')[1])

In [22]:
global_title = []
global_author = []
global_rates = []

url_base = 'https://www.goodreads.com/list/show/1.Best_Books_Ever'
for i in tqdm(range(1,101)):
    url_complete = url_base + '?page={}'.format(str(i))
    extract_page2(url_complete, global_title, global_author, global_rates)

100%|█████████████████████████████████████████| 100/100 [08:14<00:00,  4.94s/it]


In [23]:
dataframe2 = pandas.DataFrame(list(zip(global_title, global_author, global_rates)), columns=['Title', 'Author', 'Rate'])

In [32]:
dataframe2

,Title,Author,Rate
0,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,4.32
1,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling,4.50
2,To Kill a Mockingbird,Harper Lee,4.27
3,Pride and Prejudice,Jane Austen,4.27
4,"Twilight (The Twilight Saga, #1)",Stephenie Meyer,3.62
...,...,...,...
9995,I Am Princess X,Cherie Priest,4.07
9996,Hate Notes,Vi Keeland,4.14
9997,The Know-It-All: One Man's Humble Quest to Bec...,A.J. Jacobs,3.74
9998,"Twisted Pride (The Camorra Chronicles, #3)",Cora Reilly,4.25


On remet à zéro pour la suite

In [25]:
global_title = []
global_author = []
global_rates = []

# Solution plus maintenable et élégante ;)

In [42]:
class Scraper():
    
    def __init__(self):
        self.url = None
        self.page = None
        self.soup = None
        self.global_title = []
        self.global_author = []
        self.global_rates = []
        
    def set_url(self, url):
        self.url = url
        self.page = urlopen(url)
        self.soup = BeautifulSoup(self.page, 'html.parser')
        
    def extract_author_title(self):
        livre = self.soup.find_all(name = 'span', attrs={'itemprop': 'name'})
        for i, element in enumerate(livre):
            if i % 2 != 0:
                self.global_author.append(element.text)
            else:
                self.global_title.append(element.text)
    
    def extract_ratings(self):
        rating = self.soup.find_all(name = 'span', attrs={'class': 'minirating'})
        for el in rating:
            if el.text.split(' ')[0] == 'really':
                self.global_rates.append(el.text.split(' ')[3])
            else:
                self.global_rates.append(el.text.split(' ')[1])
                
    def scrap(self):
        self.extract_author_title()
        self.extract_ratings()
        
    def get_dataframe(self):
        df = pandas.DataFrame(list(zip(self.global_title, 
                                       self.global_author, 
                                       self.global_rates)), columns=['Title', 'Author', 'Rate'])
        return df
        

In [43]:
url_base = 'https://www.goodreads.com/list/show/1.Best_Books_Ever'

scraper = Scraper()

for i in tqdm(range(1,101)):
    url_complete = url_base + '?page={}'.format(str(i))
    scraper.set_url(url_complete)
    scraper.scrap()
    
dataframe3 = scrapper.get_dataframe()


100%|█████████████████████████████████████████| 100/100 [08:18<00:00,  4.99s/it]


In [44]:
dataframe3

,Title,Author,Rate
0,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,4.32
1,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling,4.50
2,To Kill a Mockingbird,Harper Lee,4.27
3,Pride and Prejudice,Jane Austen,4.27
4,"Twilight (The Twilight Saga, #1)",Stephenie Meyer,3.62
...,...,...,...
9995,I Am Princess X,Cherie Priest,4.07
9996,Hate Notes,Vi Keeland,4.14
9997,The Know-It-All: One Man's Humble Quest to Bec...,A.J. Jacobs,3.74
9998,"Twisted Pride (The Camorra Chronicles, #3)",Cora Reilly,4.25


In [45]:
dataframe3.to_csv('corrections.csv', index=False)